Code de calcul des statistiques pour article TALN 2025 (Barletta, Ponton)
Prends en entrée un fichier csv ayant cette structure (généré à partir des annotations au format UIMA CAS 1.0 et les codes merge_uimacas puis uimacas_stats_unify)


In [1]:
import pandas as pd

In [ ]:
#calcul mentions, chaines, anaphores et singletons ?


Calcul de la densité : 
nb des mentions par texte / nb de tokens par texte
Filtrage des mentions contenues dans des mentions plus larges pour ne pas biaser le calcul (ex ((son) chat) - "son" éliminé du calcul des mentions 

In [15]:
##combien de tokens dans les mentions sum tot et sum par niveau
def remove_contained_spans(df, suffix):
    kept_spans = []
    removed_spans = []

    # Group by 'Source' to process spans within the same source separately
    for source, group in df.groupby('Source'):
        # Sort spans within each group
        group_sorted = group.sort_values(by=['begin', 'end'], ascending=[True, False]).reset_index(drop=True)

        # Track kept spans and max end value
        filtered_spans = []
        max_end = -1
        
        for _, row in group_sorted.iterrows():
            begin, end = row['begin'], row['end']

            # If the span is contained within a larger span, remove it
            if end <= max_end:
                removed_spans.append(row)
            else:
                filtered_spans.append(row)
                max_end = max(max_end, end)  # Update max_end
        
        kept_spans.extend(filtered_spans)

    # Convert to DataFrame
    df_filtered = pd.DataFrame(kept_spans)
    df_removed = pd.DataFrame(removed_spans)

    # Save removed spans to CSV
    df_removed.to_csv('removed_spans'+suffix+'.csv', index=False)

    return df_filtered  # Return the cleaned DataFrame


#CALCUL DE LA DENSITE référentielle : nb tokens mentions / nb tokens texte 
def calcul_density(fr, toks_df, filename) : 
    # Filter out empty values in 'Text' column (Source)
    df_fr_filtered = fr[fr['Source'] != '']

    # Group by 'Text' and count occurrences
    df_counts = df_fr_filtered.groupby('Source').size().reset_index(name='Mentions')
    df_counts['Source'] = df_counts['Source'].str[:-4]

    #mentions_sum = df_counts['Mentions'].sum()

    # Rename 'old_name' to 'new_name'
    df_counts.rename(columns={'Source': 'texte'}, inplace=True)
    #print(df_counts)

    fr_toks = pd.read_csv(toks_df)
    fr_toks = fr_toks[['texte', 'nbTokNoPunct']]
    #corpus_francais_v3
    # Merge on the 'text' column
    df_merged = df_counts.merge(fr_toks, on='texte', how='inner')  # 'inner' ensures only matching "text" values remain

    #print(df_merged)

    #densité = ratio nb ER / nb tokens (no punct)
    df_merged['density'] = df_merged['Mentions'] / df_merged['nbTokNoPunct']*100

    df_merged.to_csv(filename, index=False)

    level="CE1"
    df_CE1 = df_merged[df_merged['texte'].str.contains(level, case=False, na=False)]
    level="CE2"
    df_CE2 = df_merged[df_merged['texte'].str.contains(level, case=False, na=False)]

    moy_density_CE1 = df_CE1['density'].mean()
    moy_density_CE2 = df_CE2['density'].mean()
    moy_density = df_merged['density'].mean()
    
    return moy_density, moy_density_CE1, moy_density_CE2

###MAIN FR
fr = pd.read_csv("./annotations_francais/toutes_mentions_corpus_detail_interdistance_fr.csv")
fr = fr.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

toks_df = "./annotations_francais/corpus_francais_v3.csv"
res_fr = "./annotations_francais/density_fr.csv"
fr_filtered = remove_contained_spans(fr, "fr")

moy_density, moy_density_CE1, moy_density_CE2 = calcul_density(fr_filtered, toks_df, res_fr)

print("*** FRANCAIS ***")

print("Densité moyenne CE1 : ", moy_density_CE1)
print("Densité moyenne CE2 : ", moy_density_CE2)
print("Densité moyenne : ", moy_density)

# ####MAIN ITA
ita = pd.read_csv("./annotations_italien/toutes_mentions_corpus_detail_interdistance_ita.csv")
ita = ita.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

toks_df = ("./annotations_italien/corpus_italien.csv")
res_ita = "./annotations_italien/density_ita.csv"
ita_filtered = remove_contained_spans(ita, "ita")

moy_density, moy_density_CE1, moy_density_CE2 = calcul_density(ita_filtered, toks_df, res_ita)

print("*** ITA ***")

print("Densité moyenne CE1 : ", moy_density_CE1)
print("Densité moyenne CE2 : ", moy_density_CE2)
print("Densité moyenne : ", moy_density)

*** FRANCAIS ***
Densité moyenne CE1 :  22.119334468721494
Densité moyenne CE2 :  21.17301431824066
Densité moyenne :  21.64617439348108
*** ITA ***
Densité moyenne CE1 :  22.123998442875674
Densité moyenne CE2 :  21.631055782214172
Densité moyenne :  21.87752711254493


calcul du coefficient de stabilité de Perret 
- filtra le catene meno lunghe di 3 (solo catene vere e proprie nel conto)
- esclude la prima menzione della catena nel caso in cui sia una menzione nominale indefinita (un gatto)
- da come risultato la stabilité di perret e l'instabilité di Rousier-Vercruyssen e Landragin 

In [19]:
file = "./annotations_francais/toutes_mentions_corpus_detail_interdistance_fr.csv"
#file = "./annotations_italien/toutes_mentions_corpus_detail_interdistance_ita.csv"

df = pd.read_csv(file)


#filtrer singletons et anaphores et inclure seulement chaines 3+
# Count occurrences of each tag in each Source
tag_counts = df.groupby(['tag', 'Source']).size().reset_index(name='count')

# Keep only tags appearing more than twice in a Source
valid_tags = tag_counts[tag_counts['count'] > 2]['tag']

# Filter the original DataFrame
filtered_df = df[df['tag'].isin(valid_tags)]

print(filtered_df)


# Filtering rows where 'Text' column contains "NOUN" --> mentions nominales (SN)
# erreurs possibles
sn_mentions = df[df['POSno'].str.contains('NOUN', na=False)]

print(sn_mentions)

#lower case to avoid upper/lower case bias
sn_mentions['mention'] = sn_mentions['mention'].str.lower()

# #TODO add filter out IndSN mentions
# # Group by 'Source' and 'tag', then compute the required metrics
# result = sn_mentions.groupby(['Source', 'tag']).agg(
#     nominalMent=('mention', 'count'),  # Count total mentions
#     unique=('mention', pd.Series.nunique)  # Count unique mentions
# ).reset_index()

# # Add the new column with the formula
# result['stability'] = result['nominalMent'] / result['unique']
# result['instability'] = result['unique'] / result['nominalMent'] * 100

# print(result)
# instability_moy = result['instability'].mean()

# #moyenne de l'instabilité par niveau puis total
# level="CE1"
# df_CE1 = result[result['Source'].str.contains(level, case=False, na=False)]
# ce1_instability_moy = df_CE1['instability'].mean()

# level="CE2"
# df_CE2 = result[result['Source'].str.contains(level, case=False, na=False)]
# ce2_instability_moy = df_CE2['instability'].mean()


###calcul du vrai coefficient de Perret : 
# si la première mention de la chaine est un SN indéfini, exclure du décompte

#group mentions by tag
#if tag_occurrences is 1 and first POSno is DET and 1st part of morphNoPunct is equal to Definite=Ind

sn_indef = sn_mentions[
    (sn_mentions['tag_occurrences'] == 1) &
    (sn_mentions['POSno'].str.startswith("['DET")) &
    (sn_mentions['morphNoPunct'].str.startswith("['Definite=Ind"))
]

print(sn_indef)

# mentions filtrées : seulement mentions nominales, exclus les première mentions de chaine SN indéfini
sn_no_indef = sn_mentions[
    ~(
      (sn_mentions['tag_occurrences'] == 1) &
      (sn_mentions['POSno'].str.startswith("['DET")) &
      (sn_mentions['morphNoPunct'].str.startswith("['Definite=Ind"))
     )
]

print(sn_no_indef)


#calcul du coefficient
result = sn_no_indef.groupby(['Source', 'tag']).agg(
    nominalMent=('mention', 'count'),  # Count total mentions
    unique=('mention', pd.Series.nunique)  # Count unique mentions
).reset_index()

# Add the new column with the formula
result['stability'] = result['nominalMent'] / result['unique']
result['instability'] = result['unique'] / result['nominalMent'] * 100

print(result)
instability_moy = result['instability'].mean()
stability_moy = result['stability'].mean()

#moyenne de l'instabilité par niveau puis total
level="CE1"
df_CE1 = result[result['Source'].str.contains(level, case=False, na=False)]
ce1_instability_moy = df_CE1['instability'].mean()
ce1_stability_moy = df_CE1['stability'].mean()

level="CE2"
df_CE2 = result[result['Source'].str.contains(level, case=False, na=False)]
ce2_instability_moy = df_CE2['instability'].mean()
ce2_stability_moy = df_CE2['stability'].mean()




txt = "./instability_fr.txt"
with open(txt, "w") as file:
    file.write(f"Nb of nominal anaphora tot.: {len(sn_mentions)}\n")
    file.write(f"Nb of nominal anaphora no Indef tot.: {len(sn_no_indef)}\n")
    
    file.write(f"Stability CE1 : {ce1_stability_moy}\n")
    file.write(f"Stability CE2 : {ce2_stability_moy}\n")
    file.write(f"Stability CE1-CE2 : {stability_moy}\n")
    
    file.write(f"Instability CE1 : {ce1_instability_moy}\n")
    file.write(f"Instability CE2 : {ce2_instability_moy}\n")
    file.write(f"Instability CE1-CE2 : {instability_moy}\n")
    
with open(txt, "r") as res:
    print(res.readlines())


      Unnamed: 0  Unnamed: 0.1  begin  end              mention       tag  \
0              0             0     18   37  un petit chat blanc       cat   
1              1             1     38   41                  qui       cat   
2              2             2     77   90        le petit chat       cat   
3              3             3    107  109                   il       cat   
4              4             4    183  202  le petit chat blanc       cat   
...          ...           ...    ...  ...                  ...       ...   
3136        3136            10    165  168                  vit      wolf   
3137        3137            11    266  273              le loup      wolf   
3138        3138            12    232  251  son ami le chasseur      ext1   
3139        3139            13    259  261                   il      ext1   
3140        3140            14    298  301                  ils  cat+ext1   

                                 POS  \
0      ['DET', 'ADJ', 'NOUN', 'ADJ'

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_28552/3355923136.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sn_mentions['mention'] = sn_mentions['mention'].str.lower()


In [20]:
import pandas as pd

# Assuming you already have the DataFrame `df` loaded

# Filtering rows for SN mentions
sn_mentions = df[df['POSno'].str.contains('NOUN', na=False)]
sn_mentions['mention'] = sn_mentions['mention'].str.lower()

# Filtering out SN mentions with 'IndSN' (if needed)
sn_no_indef = sn_mentions[~(
    (sn_mentions['tag_occurrences'] == 1) & 
    (sn_mentions['POSno'].str.startswith("['DET")) & 
    (sn_mentions['morphNoPunct'].str.startswith("['Definite=Ind"))
)]

# Grouping by 'Source' and 'tag', then calculate the stability and instability for each group
result = sn_no_indef.groupby(['Source', 'tag']).agg(
    nominalMent=('mention', 'count'),  # Total mentions
    unique=('mention', pd.Series.nunique)  # Count unique mentions
).reset_index()

# Add columns for stability and instability
result['stability'] = result['nominalMent'] / result['unique']
result['instability'] = result['unique'] / result['nominalMent'] * 100

# Merge the computed 'stability' and 'instability' back into the original DataFrame
df_with_stability = sn_no_indef.merge(result[['Source', 'tag', 'stability', 'instability']], 
                                      on=['Source', 'tag'], 
                                      how='left')

# Display the result (DataFrame with stability and instability for each row)
print(df_with_stability)

# Optionally, save the result to a CSV if you need
df_with_stability.to_csv('df_with_stability_instability.csv', index=False)


      Unnamed: 0  Unnamed: 0.1  begin  end              mention   tag  \
0              2             2     77   90        le petit chat   cat   
1              4             4    183  202  le petit chat blanc   cat   
2              5             5    220  233        le petit chat   cat   
3              8             8    139  146              le loup  wolf   
4             10            10    237  244              le loup  wolf   
...          ...           ...    ...  ...                  ...   ...   
1112        3126             0      0   17    la chatte magique   cat   
1113        3127             1     49   59           une chatte   cat   
1114        3129             3    120  132         cette chatte   cat   
1115        3137            11    266  273              le loup  wolf   
1116        3138            12    232  251  son ami le chasseur  ext1   

                                 POS  \
0             ['DET', 'ADJ', 'NOUN']   
1      ['DET', 'ADJ', 'NOUN', 'ADJ']   
2  

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_28552/2476417353.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sn_mentions['mention'] = sn_mentions['mention'].str.lower()


Creazione dell'Excel con le tipologie di menzione ATTENZIONE va corretto manualmente per evitare il tasso di errore su alcune categorie

In [ ]:
file = "./annotations_francais/toutes_mentions_corpus_detail_interdistance_fr.csv"

df = pd.read_csv(file)

# Assuming df is already defined
remaining_df = df.copy()  # Copy the original dataframe to filter progressively

# Extract each category and update remaining_df
sn_def = remaining_df[remaining_df['POSno'].str.contains('NOUN', na=False) & remaining_df['morphNoPunct'].str.startswith("['Definite=Def", na=False)]
remaining_df = remaining_df.drop(sn_def.index)  # Remove classified elements

sn_ind = remaining_df[remaining_df['POSno'].str.contains('NOUN', na=False) & remaining_df['morphNoPunct'].str.startswith("['Definite=Ind", na=False)]
remaining_df = remaining_df.drop(sn_ind.index)

#TODO add criteria
sn_poss = remaining_df[remaining_df['POSno'].str.contains('DET', na=False) & remaining_df['morphNoPunct'].str.contains("Poss=Yes", na=False)
                       & remaining_df['POSno'].str.contains('NOUN', na=False)]
remaining_df = remaining_df.drop(sn_poss.index)

verb = remaining_df[remaining_df['POSno'].str.contains('VERB|AUX', na=False) & ~remaining_df['POSno'].str.contains('NOUN', na=False)]
remaining_df = remaining_df.drop(verb.index)

pron = remaining_df[remaining_df['POSno'] == "['PRON']"]
remaining_df = remaining_df.drop(pron.index)

propn = df[(df['POSno'] == "['PROPN']") | (df['POSno'] == "['PROPN', 'PROPN']")]
remaining_df = remaining_df.drop(propn.index)

det_poss = remaining_df[(remaining_df['POSno'] == "['DET']") & remaining_df['morphNoPunct'].str.contains('Poss=Yes', na=False)]
remaining_df = remaining_df.drop(det_poss.index)

sn_dem = remaining_df[(remaining_df['POSno'].str.startswith("['DET'", na=False)) & remaining_df['morphNoPunct'].str.contains('PronType=Dem', na=False)]
remaining_df = remaining_df.drop(sn_dem.index)

num = remaining_df[(remaining_df['POSno'].str.startswith("['NUM'", na=False))]
remaining_df = remaining_df.drop(num.index)

sn_no_det = remaining_df[remaining_df['POSno'].str.contains('NOUN', na=False) & ~remaining_df['POSno'].str.startswith("['DET", na=False)]
remaining_df = remaining_df.drop(sn_no_det.index)


# Collect remaining elements into "autre"
autre = remaining_df.copy()

# Write to Excel
with pd.ExcelWriter('fr_types.xlsx') as writer:
    sn_def.to_excel(writer, sheet_name='sn_def', index=False)
    sn_ind.to_excel(writer, sheet_name='sn_indef', index=False)
    sn_poss.to_excel(writer, sheet_name='sn_poss', index=False)
    verb.to_excel(writer, sheet_name='verb', index=False)
    pron.to_excel(writer, sheet_name='pron', index=False)
    propn.to_excel(writer, sheet_name='propn', index=False)
    det_poss.to_excel(writer, sheet_name='det_poss', index=False)
    sn_dem.to_excel(writer, sheet_name='sn_dem', index=False)
    sn_no_det.to_excel(writer, sheet_name='sn_no_det', index=False)
    num.to_excel(writer, sheet_name='numerals', index=False)
    autre.to_excel(writer, sheet_name='autre', index=False)  # Write remaining elements
    
   
print("Excel file 'fr_types.xlsx' created successfully!")

In [ ]:
#file = "./annotations_francais/toutes_mentions_corpus_detail_interdistance_fr.csv"
file = "./annotations_italien/toutes_mentions_corpus_detail_interdistance_ita.csv"

df = pd.read_csv(file)

# Assuming df is already defined
remaining_df = df.copy()  # Copy the original dataframe to filter progressively

sn_poss = remaining_df[remaining_df['POSno'].str.contains('DET', na=False) & remaining_df['morphNoPunct'].str.contains("Poss=Yes", na=False)
                       & remaining_df['POSno'].str.contains('NOUN', na=False)]
remaining_df = remaining_df.drop(sn_poss.index)


propn = remaining_df[(remaining_df['POSno'] == "['PROPN']") | (remaining_df['POSno'] == "['PROPN', 'PROPN']") | remaining_df['POSno'].str.startswith("['PROPN'") ]
remaining_df = remaining_df.drop(propn.index)

# Extract each category and update remaining_df
sn_def = remaining_df[remaining_df['POSno'].str.contains('NOUN', na=False) & remaining_df['morphNoPunct'].str.startswith("['Definite=Def", na=False)]
remaining_df = remaining_df.drop(sn_def.index)  # Remove classified elements

sn_ind = remaining_df[remaining_df['POSno'].str.contains('NOUN', na=False) & remaining_df['morphNoPunct'].str.startswith("['Definite=Ind", na=False)]
remaining_df = remaining_df.drop(sn_ind.index)

#TODO add criteria
verb = remaining_df[remaining_df['POSno'].str.contains('VERB|AUX', na=False)]
remaining_df = remaining_df.drop(verb.index)

sn_dem = remaining_df[(remaining_df['POSno'].str.startswith("['DET'", na=False)) & remaining_df['morphNoPunct'].str.contains('PronType=Dem', na=False)]
remaining_df = remaining_df.drop(sn_dem.index)


pron_clit_formes = "lo|la|gli|le|li|gli|glie|mi|ti|ci|si|ne|lui|loro|che"
# Correct condition syntax
pron = remaining_df[(remaining_df['POSno'] == "['PRON']") | 
                    (remaining_df['mention'].str.contains(pron_clit_formes, na=False))]

# Drop identified rows from remaining_df
remaining_df = remaining_df.drop(pron.index)


det_poss = remaining_df[(remaining_df['POSno'] == "['DET']") & remaining_df['morphNoPunct'].str.contains('Poss=Yes', na=False)]
remaining_df = remaining_df.drop(det_poss.index)


num = remaining_df[(remaining_df['POSno'].str.startswith("['NUM'", na=False))]
remaining_df = remaining_df.drop(num.index)

sn_no_det = remaining_df[remaining_df['POSno'].str.contains('NOUN', na=False) & ~remaining_df['POSno'].str.startswith("['DET", na=False)]
remaining_df = remaining_df.drop(sn_no_det.index)


# Collect remaining elements into "autre"
autre = remaining_df.copy()

# Write to Excel
with pd.ExcelWriter('ita_types.xlsx') as writer:
    sn_def.to_excel(writer, sheet_name='sn_def', index=False)
    sn_ind.to_excel(writer, sheet_name='sn_indef', index=False)
    sn_poss.to_excel(writer, sheet_name='sn_poss', index=False)
    verb.to_excel(writer, sheet_name='verb', index=False)
    pron.to_excel(writer, sheet_name='pron', index=False)
    propn.to_excel(writer, sheet_name='propn', index=False)
    det_poss.to_excel(writer, sheet_name='det_poss', index=False)
    sn_dem.to_excel(writer, sheet_name='sn_dem', index=False)
    sn_no_det.to_excel(writer, sheet_name='sn_no_det', index=False)
    num.to_excel(writer, sheet_name='numerals', index=False)
    autre.to_excel(writer, sheet_name='autre', index=False)  # Write remaining elements
    
print("Excel file 'ita_types.xlsx' created successfully!")

calcul de la densité = nb mentions / nb tot tokens du texte
TODO calcul de la densité nb TOKENS MENTIONS / nb tokens du texte

Calcolo dei tipi di menzione posto 1 e tipi di menzione posto 2 - da fare su file di categorizzazione delle menzioni per tipo già corretto manualmente

In [11]:
import pandas as pd
file = "./annotations_francais/toutes_mentions_corpus_detail_interdistance_fr.csv"
#file = "./annotations_italien/toutes_mentions_corpus_detail_interdistance_ita.csv"


# Define a mapping of sheet names to group names
sheet_groups = {
    "sn_def": "SN défini",
    "sn_indef": "SN indéfini",
    "verb": "Anaphore Zéro",
    "pron": "Pronom",
    "propn": "Nom Propre", 
    "det_poss": "SN possessif", 
    "sn_poss": "SN possessif",
    "sn_dem" : "SN démonstratif",
    "sn_no_det" : "SN sans déterminant",
    "numerals" : "Autre",
    "autre" : 'Autre'
}


def calculate_ERType(file_path, sheet_groups, ment_order=None) :
    xls = pd.ExcelFile(file_path)
    
    # Dictionary to store dataframes
    dfs = {}
    
    # Read each sheet into a dataframe
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet_name)
        # Ensure 'Index' column exists
        if 'Unnamed: 0' in df.columns:
            dfs[sheet_name] = df  # Store dataframe
    
    total_rows = sum(len(df.dropna(how="all")) for df in dfs.values())
    print(f"Total number of non-empty rows across all sheets: {total_rows}")
    
    # Create a list to store sheet names, groups, and row counts
    sheet_info = []
    
    # Iterate through each sheet
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet_name)
        
        # Drop completely empty rows
        df = df.dropna(how="all")
        
        # If tag_value is provided, filter rows where 'tag' column matches the given value
        if ment_order is not None and "tag_occurrences" in df.columns:
            df = df[df["tag_occurrences"] == ment_order]
        
        # Count rows excluding the header
        row_count = len(df)
        
        # Determine the group (default to "Uncategorized" if not found in mapping)
        group_name = sheet_groups.get(sheet_name, "Uncategorized")
        
        # Append sheet name, group, and row count to the list
        sheet_info.append({"Sheet Name": sheet_name, "Type": group_name, "Nb": row_count})
    
    # Convert to DataFrame
    sheet_info_df = pd.DataFrame(sheet_info)
    
    # Display the DataFrame
    print(sheet_info_df)
    
    # Convert to DataFrame
    sheet_info_df = pd.DataFrame(sheet_info)
    
    # Now, group by 'Group' and sum the 'Row Count'
    grouped_df = sheet_info_df.groupby("Type")["Nb"].sum().reset_index()
    
    # Display the grouped DataFrame
    print(grouped_df)
    return grouped_df

# Load the Excel file - un foglio per tipologia di menzione
#file_path_it = "ita_types_v2.xlsx"
file_path_fr = "typologie_mentions_fr_V3_DEF.xlsx"




df = calculate_ERType(file_path_it, sheet_groups)## --> filter out only chains

sum_tout = df['Nb'].sum()
print(sum_tout)

Total number of non-empty rows across all sheets: 4829
   Sheet Name                 Type    Nb
0      sn_def            SN défini  1250
1    sn_indef          SN indéfini   397
2     sn_poss         SN possessif   100
3        verb        Anaphore Zéro  1473
4        pron               Pronom   945
5       propn           Nom Propre   323
6    det_poss         SN possessif   208
7      sn_dem      SN démonstratif    35
8   sn_no_det  SN sans déterminant    56
9    numerals                Autre     7
10      autre                Autre    35
                  Type    Nb
0        Anaphore Zéro  1473
1                Autre    42
2           Nom Propre   323
3               Pronom   945
4            SN défini  1250
5      SN démonstratif    35
6          SN indéfini   397
7         SN possessif   308
8  SN sans déterminant    56
4829


In [ ]:
##CALCUL INTERDISTANCE - taux interdistance et moyenne par niveau/langue
import pandas as pd

#VF 
file = "./toutes_mentions_corpus_detail_interdistance_fr.csv"
#VI
#file = "./toutes_mentions_corpus_detail_interdistance_ita.csv"


#file to pandas df
df = pd.read_csv(file)

#if existing, drop these cols
# Define the columns to drop
cols_to_drop = ['Unnamed: 0', 'Unnamed: 0.1']

# Drop them if they exist
df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

df = df.rename(columns={'Source': 'texte'})
# GroupBy su 'texte' e 'tag' e applicazione delle operazioni richieste
result = df.groupby(['texte', 'tag']).agg({
    'tag_occurrences': 'max',         # Trova il valore massimo di 'occurrence'
    'interdistance': 'sum',          # Somma i valori di 'interdistance'
}).reset_index()

#Filtrare solo le catene cioè tag_occurrences >= 3
result = result[(result['tag_occurrences'] >= 3)]

#taux interdistance
# Aggiunta della nuova colonna con la divisione somma interdistanza / len chaine
result['taux_interdistance'] = round(result['interdistance'] / result['tag_occurrences'],2)
mean_tot1 = result['taux_interdistance'].mean()


#risultato per ogni catena in csv
result.to_csv("./corpus_francais_interdistance.csv", ",", encoding="utf-8")

#risultato per testo - media dei tassi ottenuti per ogni catena per testo
# moyenne entre taux d'interdistance du même texte
mean_interdistance = result.groupby('texte')['taux_interdistance'].mean().reset_index()
mean_tot2 = result['taux_interdistance'].mean()

##media per livello
mean_ce1 = result.loc[result['texte'].str.contains('CE1', na=False), 'taux_interdistance'].mean()
mean_ce2 = result.loc[result['texte'].str.contains('CE2', na=False), 'taux_interdistance'].mean()

# filtered_CE1 = res.loc[res['texte'].str.contains('CE1', na=False), 'Characters'].value_counts()

In [5]:
import pandas as pd
#calcul moyenne de personnage/entités par texte (entité sing et entités plurielles - séparé)
#V1 - toute chaine
#V2 - seulement coréf (>= 3)

file = "./annotations_italien/len_stats_ita.csv"
file = "./annotations_francais/len_stats_fr.csv"
df = pd.read_csv(file)

# Step 1: Count how many values are > 0 per row (excluding 'Source' column)
df['Entities'] = (df.iloc[:, 1:] > 0).sum(axis=1)

# Step 2: Compute the mean of 'Count_Positive'
mean_positive = df['Entities'].mean()

# Step 3: Count how many values are >= 3 per row
df['Chains'] = (df.iloc[:, 1:-1] >= 3).sum(axis=1)

# Step 4: Compute the mean of 'Count_GreaterEqual_3'
mean_greater_equal_3 = df['Chains'].mean()

# Print results
print(df)
print(f"Mean of Entities: {mean_positive}")
print(f"Mean of Chains: {mean_greater_equal_3}")

                    Source  cat  cat+cat1  cat+cat1+cat2  cat+cat1+cat2+cat3  \
0     CE1-2015-102-D1-S212    6         0              0                   0   
1     CE1-2015-102-D1-S217    0         0              0                   0   
2     CE1-2015-104-D1-S824    6         0              0                   0   
3     CE1-2015-104-D1-S841    9         0              0                   0   
4    CE1-2015-108-D1-S2977    3         0              0                   0   
..                     ...  ...       ...            ...                 ...   
143   CE2-2016-96-D1-S1926   13         0              0                   0   
144    CE2-2016-98-D1-S816    0         0              0                   0   
145    CE2-2016-98-D1-S834    6         0              0                   0   
146    CE2-2016-98-D1-S936    6         0              0                   0   
147    CE2-2016-98-D1-S940    9         0              0                   0   

     cat+ext1  cat+ext1+ext2  cat+ext2 

In [15]:
import pandas as pd
#calcul moyenne de personnage/entités par texte (entité sing et entités plurielles - séparé)
#V1 - toute chaine
#V2 - seulement coréf (>= 3)

ita_file = "./annotations_italien/len_stats_ita.csv"
fr_file = "./annotations_francais/len_stats_fr.csv"

df = pd.read_csv(ita_file)

stop=col = 'wolf3'

#print(df)
#print(df.columns)

def count_ints_gt_3(df, value_to_filter, stop_column):
    # Step 1: Filter rows where 'Source' contains the value (e.g., 'CE1')
    filtered_df = df[df['Source'].str.contains(value_to_filter, na=False)]
    
    # Step 2: Find the index of the 'Source' column and then get columns after it
    source_index = df.columns.get_loc('Source')  # Get the position of 'Source'
    columns_after_source = df.columns[source_index + 1 : ]  # Columns after 'Source'
    
    # Step 3: Select columns up until and including the stop column (among columns after 'Source')
    column_index = df.columns.get_loc(stop_column) + 1
    columns_up_to_stop = [col for col in columns_after_source if df.columns.get_loc(col) < column_index]
    
    filtered_df = filtered_df[columns_up_to_stop]
    
    # Step 4: Count how many values >= 3 in each row (only considering numeric columns)
    count_gt_3 = (filtered_df >= 3).sum(axis=1)
    
    return count_gt_3

ita_chains_CE1 = count_ints_gt_3(df, 'CE1', stop)
print(len(ita_chains_CE1))
ita_chains_CE2 = count_ints_gt_3(df, 'CE2', stop)
print(len(ita_chains_CE2))

75
75
